In [17]:

from datetime import date, datetime, timedelta
from zoneinfo import ZoneInfo

today = date.today()
tzinfo = ZoneInfo('America/New_York')
now = datetime.now().astimezone()
now + timedelta(days=6, hours=18, minutes=51)

datetime.datetime(2022, 11, 13, 11, 31, 3, 736582, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'Eastern Standard Time'))


# Get list of most popular songs by year, month, and day

In [11]:

%run ../load_magic/soup.py

page_url = 'https://en.wikipedia.org/wiki/List_of_Billboard_Year-End_number-one_singles_and_albums'
table_dfs_list = get_wiki_tables(page_url)

[(0, (76, 7)), (1, (5, 2))]


In [19]:

df = table_dfs_list[0].copy().replace('—', np.nan)
df.columns = ['Year', 'Pop Single', 'Pop Album', 'R&B/Soul/Hip-hop Single', 'R&B/Soul/Hip-hop Album', 'Country Single', 'Country Album']
df['pop_single_song_name'] = df['Pop Single'].map(lambda x: str(x).split('[')[0].replace('"', '').strip())
df['pop_single_artist_name'] = df['Pop Single'].map(lambda x: str(x).split(']')[1].strip())
print(df.Year.min(), df.Year.max())
columns_list = ['Year', 'Pop Single', 'pop_single_song_name', 'pop_single_artist_name']
df[columns_list].sample(20)

1946 2021


,Year,Pop Single,pop_single_song_name,pop_single_artist_name
38,1984,"""When Doves Cry""[185] Prince",When Doves Cry,Prince
59,2005,"""We Belong Together""[311] Mariah Carey",We Belong Together,Mariah Carey
61,2007,"""Irreplaceable""[322] Beyoncé",Irreplaceable,Beyoncé
27,1973,"""Tie a Yellow Ribbon 'Round the Ole Oak Tree""[...",Tie a Yellow Ribbon 'Round the Ole Oak Tree,Tony Orlando and Dawn
43,1989,"""Look Away""[215] Chicago",Look Away,Chicago
75,2021,"""Levitating""[404] Dua Lipa",Levitating,Dua Lipa
33,1979,"""My Sharona""[155] The Knack",My Sharona,The Knack
44,1990,"""Hold On""[221] Wilson Phillips",Hold On,Wilson Phillips
3,1949,"""Riders in the Sky""[10] Vaughn Monroe Orchestra",Riders in the Sky,Vaughn Monroe Orchestra
58,2004,"""Yeah!""[305] Usher featuring Lil Jon and Ludacris",Yeah!,Usher featuring Lil Jon and Ludacris


In [20]:

%run ../load_magic/storage.py

s = Storage()
s.store_objects(Billboard_Year_End_number_one_singles_df=df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\notebooks\saves\pkl\Billboard_Year_End_number_one_singles_df.pkl


In [28]:

import webbrowser

url_prefix = 'https://en.wikipedia.org'
page_url = url_prefix + '/wiki/List_of_Billboard_number-one_singles'
page_soup = get_page_soup(page_url)
links_list = page_soup.find_all(name='a')
billboard_dict = {}
for link_soup in links_list:
    link_str = link_soup.text
    if link_str.isnumeric():
        year_number = int(link_str)
        if year_number not in billboard_dict:
            href = link_soup['href']
            billboard_dict[year_number] = url_prefix + href
            # print(dir(link_soup))
            # break

In [79]:

from datetime import datetime

# webbrowser.open(billboard_dict[1940], new=2)
date_format = '%B %d, %Y'
billboard_columns = ['issue_date', 'song_name', 'artist_name']
billboard_df = pd.DataFrame([], columns=billboard_columns)
for year in range(1940, 2023):
    tables_df_list = pd.read_html(billboard_dict[year])
    idx = sorted([(i, df.shape) for (i, df) in enumerate(tables_df_list)], key=lambda x: x[1][0]*x[1][1], reverse=True)[0][0]
    df = tables_df_list[idx].copy().replace('—', np.nan).replace('-', np.nan).replace('Chart discontinued', np.nan)
    if ('Issue date' not in df.columns):
        df.rename(columns={'Issue date[1]': 'issue_date'}, inplace=True)
    else:
        df.rename(columns={'Issue date': 'issue_date'}, inplace=True)
    df.issue_date = df.issue_date.map(lambda x: datetime.strptime(f'{x}, {year}', date_format))
    if ('Song' not in df.columns) and ('Artist(s)' not in df.columns):
        if ('Most Played Juke Box Records' not in df.columns):
            if ('Most-Played Juke Box Records' not in df.columns):
                if ('Most Played in Jukeboxes' not in df.columns):
                    if ('Top 100' not in df.columns):
                        
                        # Fill the nans from the left with fillna, then get the leftmost column
                        columns_list = ['Top 100[2]', 'Best Sellers in Stores[2]', 'Most Played by Jockeys[3]']
                        df['song_artist'] = df[columns_list].fillna(method='bfill', axis=1).iloc[:, 0]
                        
                    else:
                        df.rename(columns={'Top 100': 'song_artist'}, inplace=True)
                    df['song_name'] = df.song_artist.map(lambda x: str(x).split('"')[1].strip())
                    df['artist_name'] = df.song_artist.map(lambda x: str(x).split('"')[2].strip())
                else:
                    
                    # Fill the nans from the left with fillna, then get the leftmost column
                    columns_list = ['Most Played in Jukeboxes', 'Best Sellers in Stores', 'Most Played by Jockeys']
                    df['song_artist'] = df[columns_list].fillna(method='bfill', axis=1).iloc[:, 0]
                    
                    df['song_name'] = df.song_artist.map(lambda x: str(x).split('"')[1].strip())
                    df['artist_name'] = df.song_artist.map(lambda x: str(x).split('"')[2].strip())
            else:
                
                # Fill the nans from the left with fillna, then get the leftmost column
                columns_list = ['Most-Played Juke Box Records', 'Best-Selling Popular Retail Records', 'Records Most-Played on the Air']
                df['song_artist'] = df[columns_list].fillna(method='bfill', axis=1).iloc[:, 0]
                
                df['song_name'] = df.song_artist.map(lambda x: str(x).split('"')[1].strip())
                df['artist_name'] = df.song_artist.map(lambda x: str(x).split('"')[2].strip())
        else:
            
            # Fill the nans from the left with fillna, then get the leftmost column
            columns_list = ['Most Played Juke Box Records', 'National Best Selling Retail Records']
            df['song_artist'] = df[columns_list].fillna(method='bfill', axis=1).iloc[:, 0]
            
            df['song_name'] = df.song_artist.map(lambda x: str(x).split('"')[1].strip())
            df['artist_name'] = df.song_artist.map(lambda x: str(x).split('"')[2].strip())
    else:
        df.rename(columns={'Song': 'song_name', 'Artist(s)': 'artist_name'}, inplace=True)
    billboard_df = pd.concat([billboard_df, df[billboard_columns]], ignore_index=True)
billboard_df.song_name = billboard_df.song_name.map(lambda x: str(x).replace('"', ''))

In [84]:

billboard_df.sample(20)

,issue_date,song_name,artist_name
2194,1982-09-04,Abracadabra,Steve Miller Band
1749,1974-02-23,The Way We Were,Barbra Streisand
1304,1965-08-14,I Got You Babe,Sonny & Cher
2070,1980-04-19,Call Me,Blondie
3868,2014-10-04,All About That Bass,Meghan Trainor
1548,1970-04-18,Let It Be,The Beatles
2188,1982-07-24,Eye of the Tiger,Survivor
2349,1985-08-24,The Power of Love,Huey Lewis and the News
2054,1979-12-29,Escape (The Piña Colada Song),Rupert Holmes
1487,1969-02-15,Everyday People,Sly & the Family Stone


In [85]:

s.store_objects(Billboard_number_one_singles_dict=billboard_dict, billboard_df=billboard_df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\notebooks\saves\pkl\Billboard_number_one_singles_dict.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\notebooks\saves\pkl\billboard_df.pkl


In [94]:

import json

secrets_path = '../data/json/secrets.json'
with open(secrets_path) as f:
    secrets_dict = json.loads(f.read())
date_format = '%B %d, %Y'
motherz_birthdate = datetime.strptime(secrets_dict['motherz_birthdate'], date_format)
malcomz_birthdate = datetime.strptime(secrets_dict['malcomz_birthdate'], date_format)
mickey_joe_married = datetime.strptime(secrets_dict['mickey_joe_married'], date_format)

In [13]:

from datetime import timedelta
import humanize

motherz_teenage_firstday = motherz_birthdate.replace(year=motherz_birthdate.year+13)
motherz_teenage_lastday = motherz_birthdate.replace(year=motherz_birthdate.year+20) - timedelta(days=1)
malcomz_start_day = wsasasasasasasasasasasasasasasasasaqedwssssssss(motherz_teenage_firstday - malcomz_birthdate, suppress=[], minimum_unit='months', format='%0.0f')
malcomz_stop_day = humanize.precisedelta(motherz_teenage_lastday - malcomz_birthdate, suppress=[], minimum_unit='months', format='%0.0f')
print_str = f"Mother was born {motherz_birthdate.strftime(date_format)},"
print_str += f" so she had been a teenager for {malcomz_start_day} when Malcom was born,"
print_str += f" and he was {malcomz_stop_day} old when Mother stopped being a teenager."
print(print_str)

Mother was born September 23, 1934, so she had been a teenager for 2 years and 1 months when Malcom was born, and he was 4 years and 11 months old when Mother stopped being a teenager.


In [17]:

malcomz_ring_delta = humanize.precisedelta(mickey_joe_married - malcomz_birthdate, suppress=[], minimum_unit='months', format='%0.0f')
print_str = f"She married Dad on {mickey_joe_married.strftime(date_format)}, when Malcolm was {malcomz_ring_delta} old."
print(print_str)

She married Dad on September 13, 1955, when Malcolm was 5 years and 11 months old.


In [18]:

from datetime import date

print(['date.{}'.format(fn) for fn in dir(date) if not fn.startswith('_')])
print(['humanize.{}'.format(fn) for fn in dir(humanize) if not fn.startswith('_')])

['date.ctime', 'date.day', 'date.fromisocalendar', 'date.fromisoformat', 'date.fromordinal', 'date.fromtimestamp', 'date.isocalendar', 'date.isoformat', 'date.isoweekday', 'date.max', 'date.min', 'date.month', 'date.replace', 'date.resolution', 'date.strftime', 'date.timetuple', 'date.today', 'date.toordinal', 'date.weekday', 'date.year']
['humanize.activate', 'humanize.apnumber', 'humanize.clamp', 'humanize.deactivate', 'humanize.filesize', 'humanize.fractional', 'humanize.i18n', 'humanize.importlib_metadata', 'humanize.intcomma', 'humanize.intword', 'humanize.naturaldate', 'humanize.naturalday', 'humanize.naturaldelta', 'humanize.naturalsize', 'humanize.naturaltime', 'humanize.number', 'humanize.ordinal', 'humanize.precisedelta', 'humanize.scientific', 'humanize.thousands_separator', 'humanize.time']


In [40]:

humanize.intcomma(humanize.apnumber(1_000))

'1,000'

In [43]:

humanize.intcomma(humanize.apnumber(9))

'nine'

In [72]:

# Return a natural representation of a timedelta or number of seconds.
from datetime import timedelta

humanize.precisedelta(timedelta(minutes=(140+30)/11.5))

'14 minutes and 46.96 seconds'

In [63]:

# Return a natural representation of a time in a resolution that makes sense.

humanize.naturaltime(timedelta(minutes=120))

'2 hours ago'

In [52]:

# Return a natural day.
# For date values that are tomorrow, today or yesterday compared to
# present day return representing string. Otherwise, return a string
# formatted according to `format`.
from datetime import timedelta

date_obj = date.today() - timedelta(days=1)
humanize.naturalday(date_obj)

'yesterday'

In [49]:

# Like `naturalday`, but append a year for dates more than ~five months away.
from datetime import timedelta

date_obj = date.today() + timedelta(days=6*30)
humanize.naturaldate(date_obj)

'Oct 06 2022'

In [76]:

print(['humanize.time.{}'.format(fn) for fn in dir(humanize.time) if not fn.startswith('_')])

['humanize.time.Enum', 'humanize.time.Unit', 'humanize.time.dt', 'humanize.time.intcomma', 'humanize.time.math', 'humanize.time.naturaldate', 'humanize.time.naturalday', 'humanize.time.naturaldelta', 'humanize.time.naturaltime', 'humanize.time.precisedelta', 'humanize.time.total_ordering']


In [77]:

humanize.time.total_ordering?

Signature: humanize.time.total_ordering(cls)
Docstring: Class decorator that fills in missing ordering methods
File:      c:\programdata\anaconda3\lib\functools.py
Type:      function


In [32]:

print(['humanize.filesize.{}'.format(fn) for fn in dir(humanize.filesize) if not fn.startswith('_')])

['humanize.filesize.naturalsize', 'humanize.filesize.suffixes']


In [31]:

print(['humanize.i18n.{}'.format(fn) for fn in dir(humanize.i18n) if not fn.startswith('_')])

['humanize.i18n.activate', 'humanize.i18n.deactivate', 'humanize.i18n.get_translation', 'humanize.i18n.gettext_module', 'humanize.i18n.local', 'humanize.i18n.os', 'humanize.i18n.thousands_separator']


In [37]:

print(['humanize.importlib_metadata.{}'.format(fn) for fn in dir(humanize.importlib_metadata) if not fn.startswith('_')])

['humanize.importlib_metadata.ConfigParser', 'humanize.importlib_metadata.Distribution', 'humanize.importlib_metadata.DistributionFinder', 'humanize.importlib_metadata.EntryPoint', 'humanize.importlib_metadata.FastPath', 'humanize.importlib_metadata.FileHash', 'humanize.importlib_metadata.MetaPathFinder', 'humanize.importlib_metadata.MetadataPathFinder', 'humanize.importlib_metadata.PackageNotFoundError', 'humanize.importlib_metadata.PackagePath', 'humanize.importlib_metadata.PathDistribution', 'humanize.importlib_metadata.Prepared', 'humanize.importlib_metadata.abc', 'humanize.importlib_metadata.collections', 'humanize.importlib_metadata.csv', 'humanize.importlib_metadata.distribution', 'humanize.importlib_metadata.distributions', 'humanize.importlib_metadata.email', 'humanize.importlib_metadata.entry_points', 'humanize.importlib_metadata.files', 'humanize.importlib_metadata.functools', 'humanize.importlib_metadata.import_module', 'humanize.importlib_metadata.io', 'humanize.importlib_

In [35]:

humanize.i18n.thousands_separator()

','

In [67]:

print(['humanize.number.{}'.format(fn) for fn in dir(humanize.number) if not fn.startswith('_')])

['humanize.number.Fraction', 'humanize.number.NS_', 'humanize.number.P_', 'humanize.number.apnumber', 'humanize.number.clamp', 'humanize.number.fractional', 'humanize.number.human_powers', 'humanize.number.intcomma', 'humanize.number.intword', 'humanize.number.math', 'humanize.number.ordinal', 'humanize.number.powers', 'humanize.number.re', 'humanize.number.scientific', 'humanize.number.thousands_separator']


In [69]:

humanize.number.scientific?

Signature: humanize.number.scientific(value, precision=2)
Docstring:
Return number in string scientific notation z.wq x 10ⁿ.

Examples:
    ```pycon
    >>> scientific(float(0.3))
    '3.00 x 10⁻¹'
    >>> scientific(int(500))
    '5.00 x 10²'
    >>> scientific(-1000)
    '1.00 x 10⁻³'
    >>> scientific(1000, 1)
    '1.0 x 10³'
    >>> scientific(1000, 3)
    '1.000 x 10³'
    >>> scientific("99")
    '9.90 x 10¹'
    >>> scientific("foo")
    'foo'
    >>> scientific(None) is None
    True

    ```

Args:
    value (int, float, str): Input number.
    precision (int): Number of decimal for first part of the number.

Returns:
    str: Number in scientific notation z.wq x 10ⁿ.
File:      c:\programdata\anaconda3\lib\site-packages\humanize\number.py
Type:      function


In [17]:

# print(['datetime.{}'.format(fn) for fn in dir(datetime) if not fn.startswith('_')])
def how_long_its_been(from_date_str, to_date_str=None, suppress=[]):
    from_datetime_obj = datetime.strptime(from_date_str, '%B %d, %Y')
    if to_date_str is None:
        to_datetime_obj = datetime.now()
    else:
        to_datetime_obj = datetime.strptime(to_date_str, '%B %d, %Y')
    how_long_str = humanize.precisedelta(to_datetime_obj - from_datetime_obj, suppress=suppress,
                                         minimum_unit='days', format='%0.0f')
    
    return how_long_str

In [22]:

secs_float = 819.4461708068848
humanize.precisedelta(secs_float, minimum_unit='minutes', format='%0.0f')

'14 minutes'

In [18]:

starting_date = 'January 1, 2011'
ending_date = 'October 31, 2011'
how_long_its_been(starting_date, ending_date, suppress=['months'])

'303 days'

In [19]:

marriage_date = 'June 14, 1986'
how_long_its_been(marriage_date)

'35 years, 10 months and 1 days'

In [20]:

steves_birth_date = 'November 5, 1990'
how_long_its_been(steves_birth_date)

'31 years, 5 months and 8 days'

In [21]:

jays_birth_date = 'September 30, 1996'
how_long_its_been(jays_birth_date)

'25 years, 6 months and 12 days'

In [19]:

from datetime import timedelta

date.today() - timedelta(days=3*365)

datetime.date(2019, 4, 2)